In [55]:
from openai import OpenAI
from tqdm import tqdm
openai_api_key = "ANY THING"
openai_api_base = "http://xunziallm.njau.edu.cn:21180/v1"
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

for i in tqdm(range(0,1)):
    chat_response = client.chat.completions.create(
        model="/home/gpu0/xunzi_web/Xunzi-Qwen1.5-7B_chat",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": '翻译成白话文\n道可道非常道，名可名非常名'},
        ]
    )
    print(chat_response.choices[0].message.content)

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  1.96it/s]

道能道出的不一定是道，能说出的不一定是名。


In [26]:
## 构建client
import configparser
from openai import OpenAI

# 读取配置文件
config = configparser.ConfigParser()
config.read('config.ini')
api_prefix='xai' # 'sk'
model_prefix = 'grok'
client = OpenAI(
    api_key=config['openai']['api_key_' + api_prefix],
    base_url=config['openai']['api_base_' + api_prefix],
)

### Chat

In [27]:

## 简单聊天
model_prefix = 'grok'
content = '你是谁'
chat_response = client.chat.completions.create(
    model=config['openai']['model_' + model_prefix],
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": content},
    ]
)

chat_response.choices[0].message.content

'我是你的助手，一个由xAI创建的人工智能。我在这里帮助你回答问题，提供见解，并协助你完成各种任务。我的设计灵感来自于《指环王》中的角色，正如我的名字所示，我随时准备协助并提供智慧的建议。今天有什么可以帮助你的吗？'

### embedding

In [28]:
## 句子嵌入
model_prefix = 'grok_e'
## 生成embedding
content = ['你是谁']
chat_response = client.embeddings.create(
    model = config['openai']['model_' + model_prefix],
    input = content
)

chat_response.data[0].embedding[:3]

[-0.031065668910741806, -0.0006569150718860328, -0.04748952388763428]

### image

In [30]:
## 图片转base64编码，用于本地传输
import base64

def image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
    return encoded_string

import configparser
from openai import OpenAI

# 读取配置文件
config = configparser.ConfigParser()
config.read('config.ini')
api_prefix='sk' # 'sk'
client = OpenAI(
    api_key=config['openai']['api_key_' + api_prefix],
    base_url=config['openai']['api_base_' + api_prefix],
)

In [ ]:
## 图片聊天 client.chat.completions.create

# 将本地图片转换为Base64编码
image_path = "../data/img/daxue.jpg"

## 请求
chat_response = client.chat.completions.create(
    model="Qwen/Qwen2-VL-72B-Instruct",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        # "url": "https://sf-maas-uat-prod.oss-cn-shanghai.aliyuncs.com/dog.png" # 使用云盘 url
                        "url": f"data:image/png;base64,{image_to_base64(image_path)}"  # 使用Base64编码的图片
                    }
                },
                {
                    "type": "text",
                    "text": "描述一下图片.判断一下当时的天气，介绍图片中的景点，如果有的话。以此图片编写一个故事"
                }
            ]
        }],
)

print(chat_response.choices[0].message.content)

In [33]:
#图像生成 client.images.generate
from PIL import Image
import io,requests

# 图片生成请求
prompt = "realistic painting, featuring anthropomorphic cats and \
    dogs playing soccer on a grassy field. In the background, \
    there are distant mountains, and the sky is bright and sunny. \
    The grass should appear green, the sky blue, \
    and the sunlight should create dappled shadows on the grass."

response = client.images.generate(
    model = "stabilityai/stable-diffusion-3-5-large",
    # model='black-forest-labs/FLUX.1-dev',#"stabilityai/stable-diffusion-3-5-large",
    prompt=prompt,
    n=1,
    size="1024x1024",
    response_format="url"
)

# 解码生成的图片
image_url = response.data[0].url
# image_bytes = base64.b64decode(image_data)
# image = Image.open(io.BytesIO(image_bytes))
response = requests.get(image_url)
image = Image.open(io.BytesIO(response.content))

# 显示图片
image.save('../data/img/generate_03.jpg')
image.show()


### audio-speech

In [61]:
import configparser
from openai import OpenAI

# 读取配置文件
config = configparser.ConfigParser()
config.read('config.ini')
api_prefix='sk' # 'sk'
client = OpenAI(
    api_key=config['openai']['api_key_' + api_prefix],
    base_url=config['openai']['api_base_' + api_prefix],
)

In [4]:
## 文字转语音 client.audio.speech.with_streaming_response.create
speech_file_path = "../data/audio/generated_speech_sk_01.mp3"

with client.audio.speech.with_streaming_response.create(
  model="fishaudio/fish-speech-1.4", # 目前仅支持 fishaudio 模型
  voice="fishaudio/fish-speech-1.4:alex", # 系统预置音色
  # 用户输入信息
  input="SiliconCloud 上提供的fish audio模型是基于 70 万小时多语言音频数据训练的领先文本到语音（TTS）模型，支持中文、英语、日语、德语、法语、西班牙语、韩语、阿拉伯语等多种语言，并能够音色克隆，具有非常好的实时性。",
  response_format="mp3" # 支持 mp3, wav, pcm, opus 格式
) as response:
    response.stream_to_file(speech_file_path)



PermissionDeniedError: Error code: 403 - {'code': 30011, 'message': 'The selected model requires paid balance. Your paid balance is insufficient. Please top up and try again.', 'data': None}

In [63]:
# 语音文件路径
audio_file_path = "../data/audio/qilixiang.mp3"
# audio_file_path = "../data/audio/i-know-kung-fu.mp3"
prompt=''
# 打开音频文件
with open(audio_file_path, "rb") as audio_file:
    # 使用OpenAI的语音转文本功能
    response = client.audio.transcriptions.create(
        model="FunAudioLLM/SenseVoiceSmall",
        file=audio_file,
        prompt=prompt,
        response_format="json" # `json`, `text`, `srt`,`verbose_json`, or `vtt`.
    )

# 打印识别结果
print(response.text) #🎼窗外的麻雀在电香杆上独嘴，你说的一句很有夏天的感觉。😔手中的铅笔在纸上来来回回，我用几样子形容你是我的谁。😊
# response

🎼窗外的麻雀在电香杆上独嘴，你说的一句很有夏天的感觉。😔手中的铅笔在纸上来来回回，我用几样子形容你是我的谁。😊


In [ ]:

# 语音文件路径
audio_file_path = "../data/audio/test.mp3"
# audio_file_path = "../data/audio/i-know-kung-fu.mp3"
prompt = ''

# 定义每个片段的大小（以字节为单位）
chunk_size = 1024 * 1024  # 1MB

# 读取音频文件并切分
with open(audio_file_path, "rb") as audio_file:
    audio_data = audio_file.read()

chunks = [audio_data[i:i + chunk_size] for i in range(0, len(audio_data), chunk_size)]

# 存储所有片段的转录结果
transcriptions = []

# 依次发送每个片段的请求
for chunk in chunks:
    with open('temp_chunk.mp3', 'wb') as temp_file:
        temp_file.write(chunk)
    
    with open('temp_chunk.mp3', "rb") as temp_audio_file:
        response = client.audio.transcriptions.create(
            model="FunAudioLLM/SenseVoiceSmall",
            file=temp_audio_file,
            prompt=prompt,
            response_format="json"
        )
    
    transcriptions.append(response.text)

for line in transcriptions:
    print(line)

In [68]:
with open('speech2text_test.txt', 'w') as temp_file:
        for line in transcriptions:
            temp_file.write(line)

## test

In [70]:
import ffmpeg

# 指定输入视频文件路径
input_video_path = "input_video.mp4"

# 指定输出音频文件路径
output_audio_path = "output_audio.mp3"

# 使用 ffmpeg 提取音频
ffmpeg.input(input_video_path).output(output_audio_path, q=0, map='a').run(overwrite_output=True)

In [2]:
import os
import ffmpeg
# 指定文件夹路径
folder_path = "E:/告别原生家庭直通/tmp"
# 获取文件夹中的所有文件
all_files = os.listdir(folder_path)
# 过滤出 MP4 文件
mp4_files = [file for file in all_files if file.lower().endswith('.mp4')]

for mp4_file in mp4_files:
    file_name, _ = os.path.splitext(mp4_file)
    mp3_file_name = folder_path + '/audio_' + file_name + '.mp3'
    print('transleting {}'.format(file_name))
    ffmpeg.input(folder_path+'/'+mp4_file).output(mp3_file_name, q=0, map='a').run(overwrite_output=True)
    print('saved to {}'.format(mp3_file_name))

transleting 第1讲-从察觉原生家庭的问题到自我成长的意识
saved to E:/告别原生家庭直通/tmp/audio_第1讲-从察觉原生家庭的问题到自我成长的意识.mp3
